# Import openai library

In [ ]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install -q multiprocess timeout_decorator pebble

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00


In [ ]:
!gdown 1T-OGrjWLdDeaUBGDCA14wghW06vzU7KN

Downloading...
From (original): https://drive.google.com/uc?id=1T-OGrjWLdDeaUBGDCA14wghW06vzU7KN
From (redirected): https://drive.google.com/uc?id=1T-OGrjWLdDeaUBGDCA14wghW06vzU7KN&confirm=t&uuid=7b1513ce-f626-4f91-83c6-beb163f350ea
To: /content/python_executor.py
100% 6.31k/6.31k [00:00<00:00, 15.2MB/s]


# Init conversation

In [ ]:
from openai import OpenAI
import os


def get_completion(prompt, params):
    messages = [{"role": "user", "content":prompt}]
    client = OpenAI(
      api_key=os.environ.get("OPENAI_API_KEY"),
    )
    response = client.chat.completions.create(
        **params,
        messages=messages
    )

    return response.choices[0].message.content


os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

modelName = 'gpt-3.5-turbo'
temperature = 0.7
top_p = 1

params = {
    "model": modelName,
    "temperature": temperature,
    "top_p": top_p
}

In [ ]:
# Chain of Thought
prompt = '''
Q: An có 5 quả trứng, an mua thêm 2 hộp trứng, mỗi hộp có 6 quả, Hỏi An có bao nhiêu quả trứng.
A: Số quả trứng an có là 5 + 2 * 6 = 17 quả
Q: Một cửa hàng có 15 chiếc bánh. Buổi sáng họ bán được 3 cái, buổi chiều họ bán được 4 cái. Sau khi mua xong, có 2 chiếc bánh được trả lại. Đến tối có 4 chiếc bánh bị hư. Hỏi cửa hàng còn lại bao nhiêu cái bánh.
A:
'''
response = get_completion(prompt, params)
print(response)

Số bánh cửa hàng còn lại sau buổi sáng là 15 - 3 = 12 chiếc
Sau buổi chiều, số bánh còn lại là 12 - 4 = 8 chiếc
Sau khi trả lại và bánh bị hư, cửa hàng còn lại 8 - 2 - 4 = 2 chiếc bánh.


In [ ]:
from python_executor import PythonExecutor

# Program-aid language model
prompt = '''
Q: An có 5 quả trứng, an mua thêm 2 hộp trứng, mỗi hộp có 6 quả, Hỏi An có bao nhiêu quả trứng.
A:
orginal = 5
n_egg_boxes = 2
each_egg_on_box = 6
bought = each_egg_on_box * n_egg_boxes
total = orginal + bought
print(total)
Q: Một cửa hàng có 15 chiếc bánh. Buổi sáng họ bán được 3 cái, buổi chiều họ bán được 4 cái. Sau khi mua xong, có 2 chiếc bánh được trả lại. Đến tối có 4 chiếc bánh bị hư. Hỏi cửa hàng còn lại bao nhiêu cái bánh.
A:
'''
response = get_completion(prompt, params)
executor = PythonExecutor(get_answer_from_stdout=True)
predictions = executor.apply(response)
print(response)
print('-------------------------------------------------------')
print('Đáp án của câu hỏi là: ' + predictions[0])

final_prompt = f'''
Q: Một cửa hàng có 15 chiếc bánh. Buổi sáng họ bán được 3 cái, buổi chiều họ bán được 4 cái. Sau khi mua xong, có 2 chiếc bánh được trả lại. Đến tối có 4 chiếc bánh bị hư. Hỏi cửa hàng còn lại bao nhiêu cái bánh.
Cho biết đáp án là {predictions[0]}
A:
'''
response = get_completion(final_prompt, params)
print('-------------------------------------------------------')
print(response)

morning_sold = 3
afternoon_sold = 4
returned = 2
damaged = 4
total = 15 - morning_sold - afternoon_sold + returned - damaged
print(total)
Đáp án của câu hỏi là: 6
-------------------------------------------------------
Để giải bài toán này, ta sẽ tính số bánh còn lại sau từng bước:

- Ban đầu: 15 chiếc bánh
- Buổi sáng bán được 3 chiếc, còn lại 15 - 3 = 12 chiếc
- Buổi chiều bán được 4 chiếc, còn lại 12 - 4 = 8 chiếc
- Trả lại 2 chiếc, còn lại 8 + 2 = 10 chiếc
- Tối có 4 chiếc bánh bị hư, còn lại 10 - 4 = 6 chiếc

Vậy sau tất cả các bước trên, cửa hàng còn lại 6 chiếc bánh.
